In [1]:
import os
import pandas as pd
import requests
import time
import json
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import seaborn as sns 


from sklearn.cross_decomposition import PLSRegression
from pygam import LinearGAM, s
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
# --- CONFIG ---
API_KEY = "YwnbHRjcJvf6Md2OPoKbSRGHlzZ7hjR6"
data_folder = os.path.join(os.path.expanduser("~/Desktop/Trading"), "Data")
os.makedirs(data_folder, exist_ok=True)
tickers_csv_file = os.path.join(data_folder, "sp500_tickers.csv")

# --- LOAD FROM CACHE OR FETCH ---
if os.path.exists(tickers_csv_file):
    print("Loading tickers from CSV cache...")
    df_sp500 = pd.read_csv(tickers_csv_file)
else:
    print("Fetching tickers from API...")
    url = f"https://financialmodelingprep.com/api/v3/sp500_constituent?apikey={API_KEY}"
    df_sp500 = pd.DataFrame(requests.get(url).json())

    # Save to CSV
    df_sp500.to_csv(tickers_csv_file, index=False)
    print(f"Saved {len(df_sp500)} tickers to CSV cache.")

    
tickers = df_sp500["symbol"].dropna().unique().tolist()
# --- PREVIEW ---
print(df_sp500.shape)
print(df_sp500.columns)
print(df_sp500.head(5))

Loading tickers from CSV cache...
(503, 8)
Index(['symbol', 'name', 'sector', 'subSector', 'headQuarter',
       'dateFirstAdded', 'cik', 'founded'],
      dtype='object')
  symbol                  name                  sector  \
0    XYZ           Block, Inc.              Technology   
1    TTD  The Trade Desk, Inc.              Technology   
2   DDOG               Datadog              Technology   
3   COIN       Coinbase Global      Financial Services   
4   DASH              DoorDash  Communication Services   

                            subSector              headQuarter dateFirstAdded  \
0           Software - Infrastructure      Oakland, California     2025-07-23   
1              Software - Application      Ventura, California     2025-07-18   
2              Software - Application  New York City, New York     2025-07-09   
3  Financial - Data & Stock Exchanges     Wilmington, Delaware     2025-05-19   
4      Internet Content & Information        San Francisco, CA     2025-03

In [3]:
# --- CONFIG ---
API_KEY = "YwnbHRjcJvf6Md2OPoKbSRGHlzZ7hjR6"
data_folder = os.path.join(os.path.expanduser("~/Desktop/Trading"), "Data")
os.makedirs(data_folder, exist_ok=True)
tickers_csv_file = os.path.join(data_folder, "sp500_tickers.csv")

# Load tickers
df_sp500 = pd.read_csv(tickers_csv_file)
tickers = df_sp500["symbol"].dropna().unique().tolist()

# Output file
output_file = os.path.join(data_folder, "price_and_earnings.json")

def fetch_price_and_earnings(tickers, output_file):
    if os.path.exists(output_file):
        print(f"Loading from cache: {output_file}")
        return pd.DataFrame(json.load(open(output_file)))

    records = []
    for ticker in tickers:
        try:
            # Get current price
            quote_url = f"https://financialmodelingprep.com/api/v3/quote/{ticker}?apikey={API_KEY}"
            price_data = requests.get(quote_url).json()
            if not price_data:
                continue
            price = price_data[0]["price"]

            # Get latest annual income statement (EPS or netIncome)
            income_url = f"https://financialmodelingprep.com/api/v3/income-statement/{ticker}?limit=1&apikey={API_KEY}"
            income_data = requests.get(income_url).json()
            if not income_data:
                continue
            eps = income_data[0].get("eps")
            net_income = income_data[0].get("netIncome")

            records.append({
                "symbol": ticker,
                "price": price,
                "eps": eps,
                "netIncome": net_income
            })

        except Exception as e:
            print(f"Error fetching {ticker}: {e}")
        time.sleep(0.2)  # polite rate limit

    # Save to JSON
    with open(output_file, "w") as f:
        json.dump(records, f, indent=2)
    print(f"Saved {len(records)} records to {output_file}")

    return pd.DataFrame(records)

price_earnings_df = fetch_price_and_earnings(tickers, output_file)

Loading from cache: /Users/nicholassanso/Desktop/Trading/Data/price_and_earnings.json


In [4]:
# Only keep rows with positive EPS
pe_data = price_earnings_df[price_earnings_df["eps"] > 0].copy()

# Compute log(PE)
pe_data["log_PE"] = np.log(pe_data["price"] / pe_data["eps"])

# Optional: store just the series if you want
pe_series = pe_data["log_PE"]

# Print row count for reference
print(f"Rows used: {len(pe_series)}")
print(pe_series.head())


Rows used: 480
0    2.741115
1    4.033134
2    5.514162
3    3.434049
4    6.756855
Name: log_PE, dtype: float64


In [5]:
def fetch_statement(endpoint, tickers, period, limit, data_folder):
    """Fetch statements with unique JSON filename based on endpoint, period, limit."""
    output_file = os.path.join(
        data_folder,
        f"{endpoint}_{period}_limit{limit}.json"
    )

    if os.path.exists(output_file):
        print(f"Loading from cache: {output_file}")
        with open(output_file, "r") as f:
            return json.load(f)

    records = []
    for ticker in tickers:
        url = f"https://financialmodelingprep.com/api/v3/{endpoint}/{ticker}?period={period}&limit={limit}&apikey={API_KEY}"
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            if data:
                for row in data:
                    row["symbol"] = ticker
                records.extend(data)
        except Exception as e:
            print(f"Error fetching {ticker} ({endpoint}): {e}")
        time.sleep(.2)  # API polite rate limit

    with open(output_file, "w") as f:
        json.dump(records, f, indent=2)
    print(f"Saved {len(records)} records to {output_file}")
    return records


income_data_2_years   = fetch_statement("income-statement", tickers, "annual", 2, data_folder)
balance_data_2_years  = fetch_statement("balance-sheet-statement", tickers, "annual", 2, data_folder)
cashflow_data_2_years = fetch_statement("cash-flow-statement", tickers, "annual", 2, data_folder)

# Convert to DataFrames
income_data_2_years   = pd.DataFrame(income_data_2_years)
balance_data_2_years  = pd.DataFrame(balance_data_2_years)
cashflow_data_2_years = pd.DataFrame(cashflow_data_2_years)

print("Income shape:", income_data_2_years.shape)
print("Balance shape:", balance_data_2_years.shape)
print("Cash flow shape:", cashflow_data_2_years.shape)

print(income_data_2_years.columns)
print(balance_data_2_years.columns)
print(cashflow_data_2_years.columns)

Loading from cache: /Users/nicholassanso/Desktop/Trading/Data/income-statement_annual_limit2.json
Loading from cache: /Users/nicholassanso/Desktop/Trading/Data/balance-sheet-statement_annual_limit2.json
Loading from cache: /Users/nicholassanso/Desktop/Trading/Data/cash-flow-statement_annual_limit2.json
Income shape: (1006, 38)
Balance shape: (1006, 54)
Cash flow shape: (1006, 40)
Index(['date', 'symbol', 'reportedCurrency', 'cik', 'fillingDate',
       'acceptedDate', 'calendarYear', 'period', 'revenue', 'costOfRevenue',
       'grossProfit', 'grossProfitRatio', 'researchAndDevelopmentExpenses',
       'generalAndAdministrativeExpenses', 'sellingAndMarketingExpenses',
       'sellingGeneralAndAdministrativeExpenses', 'otherExpenses',
       'operatingExpenses', 'costAndExpenses', 'interestIncome',
       'interestExpense', 'depreciationAndAmortization', 'ebitda',
       'ebitdaratio', 'operatingIncome', 'operatingIncomeRatio',
       'totalOtherIncomeExpensesNet', 'incomeBeforeTax',
  

In [6]:
def sort_by_symbol_date(df):
    return df.sort_values(["symbol", "date"])


def compute_yoy_growth(df, exclude_cols=["symbol", "date","link","finalLink"]):
    numeric_cols = df.select_dtypes(include=[float, int]).columns
    numeric_cols = [c for c in numeric_cols if c not in exclude_cols]
    
    df_growth = df.copy()
    for col in numeric_cols:
        df_growth[col + "_yoy"] = df.groupby("symbol")[col].pct_change()
    
    return df_growth


income_sorted = sort_by_symbol_date(income_data_2_years)
balance_sorted = sort_by_symbol_date(balance_data_2_years)
cashflow_sorted = sort_by_symbol_date(cashflow_data_2_years)

income_growth = compute_yoy_growth(income_sorted)
balance_growth = compute_yoy_growth(balance_sorted)
cashflow_growth = compute_yoy_growth(cashflow_sorted)

In [7]:
def count_zeros_nans_yoy(df):
    # Keep only numeric columns that end with "_yoy"
    numeric_cols = [c for c in df.select_dtypes(include=[float, int]).columns if c.endswith("_yoy")]
    
    # Count zeros and NaNs
    zero_counts = (df[numeric_cols] == 0).sum()
    nan_counts = df[numeric_cols].isna().sum()
    
    # Combine into a single DataFrame
    summary = pd.DataFrame({
        "zeros": zero_counts,
        "nans": nan_counts
    }).sort_values(by=["zeros", "nans"], ascending=True)
    
    # Force full display
    with pd.option_context("display.max_rows", None, "display.max_columns", None):
        print(summary)
    
    return summary

# Example usage
print("Income 0/NaN counts per YoY column:")
income_summary = count_zeros_nans_yoy(income_growth)

print("\nBalance 0/NaN counts per YoY column:")
balance_summary = count_zeros_nans_yoy(balance_growth)

print("\nCashflow 0/NaN counts per YoY column:")
cashflow_summary = count_zeros_nans_yoy(cashflow_growth)

Income 0/NaN counts per YoY column:
                                             zeros  nans
revenue_yoy                                      0   503
grossProfit_yoy                                  0   503
costAndExpenses_yoy                              0   503
ebitdaratio_yoy                                  0   503
operatingIncome_yoy                              0   503
operatingIncomeRatio_yoy                         0   503
incomeBeforeTax_yoy                              0   503
incomeBeforeTaxRatio_yoy                         0   503
netIncomeRatio_yoy                               0   503
ebitda_yoy                                       0   504
operatingExpenses_yoy                            0   507
costOfRevenue_yoy                                0   510
totalOtherIncomeExpensesNet_yoy                  0   535
generalAndAdministrativeExpenses_yoy             0   756
netIncome_yoy                                    1   503
eps_yoy                                          1  

In [8]:
def clean_yoy_columns(df, nan_threshold=503, keep_cols=["symbol","date"]):
    """Select _yoy numeric columns, replace inf, drop rows/columns with too many NaNs."""
    yoy_cols = [c for c in df.select_dtypes(include=[float, int]).columns if c.endswith("_yoy")]
    sub_df = df[yoy_cols + keep_cols].copy()  # Keep symbol/date columns
    
    # Drop columns with too many NaNs (only apply to yoy columns)
    valid_yoy_cols = [c for c in yoy_cols if sub_df[c].isna().sum() <= nan_threshold]
    sub_df = sub_df[valid_yoy_cols + keep_cols]
    
    # Replace infinities with NaN and drop rows with any remaining NaNs (only apply to yoy columns)
    sub_df[valid_yoy_cols] = sub_df[valid_yoy_cols].replace([np.inf, -np.inf], np.nan)
    sub_df.dropna(subset=valid_yoy_cols, inplace=True)
    
    return sub_df


In [9]:
# Step 0: Clean _yoy columns
income_clean = clean_yoy_columns(income_growth)
balance_clean = clean_yoy_columns(balance_growth)
cashflow_clean = clean_yoy_columns(cashflow_growth)


In [10]:
# Step 1: Keep only common (symbol, date) pairs
for df in [income_clean, balance_clean, cashflow_clean]:
    df["symbol_date"] = list(zip(df["symbol"], df["date"]))

common_pairs = (
    set(income_clean["symbol_date"])
    & set(balance_clean["symbol_date"])
    & set(cashflow_clean["symbol_date"])
)

income_clean = income_clean[income_clean["symbol_date"].isin(common_pairs)].copy()
balance_clean = balance_clean[balance_clean["symbol_date"].isin(common_pairs)].copy()
cashflow_clean = cashflow_clean[cashflow_clean["symbol_date"].isin(common_pairs)].copy()

In [11]:
# Step 2: Sort consistently
income_clean = income_clean.sort_values(["symbol","date"]).reset_index(drop=True)
balance_clean = balance_clean.sort_values(["symbol","date"]).reset_index(drop=True)
cashflow_clean = cashflow_clean.sort_values(["symbol","date"]).reset_index(drop=True)

print("Cleaned and aligned shapes:", income_clean.shape, balance_clean.shape, cashflow_clean.shape)

Cleaned and aligned shapes: (491, 18) (491, 13) (491, 13)


In [12]:
income_clean.head(1)

,revenue_yoy,grossProfit_yoy,grossProfitRatio_yoy,costAndExpenses_yoy,ebitdaratio_yoy,operatingIncome_yoy,operatingIncomeRatio_yoy,incomeBeforeTax_yoy,incomeBeforeTaxRatio_yoy,netIncome_yoy,netIncomeRatio_yoy,eps_yoy,epsdiluted_yoy,weightedAverageShsOut_yoy,weightedAverageShsOutDil_yoy,symbol,date,symbol_date
0,0.02022,0.068195,0.047024,-0.004331,0.049055,0.077996,0.056631,0.085716,0.064198,-0.0336,-0.052753,-0.008117,-0.008157,-0.025435,-0.025578,AAPL,2024-09-28,"(AAPL, 2024-09-28)"


In [13]:
# Step 1: Clean & align by (symbol, date)
def clean_and_align(dfs, nan_threshold=503, keep_cols=["symbol","date"]):
    """
    1) Drop columns with too many NaNs
    2) Replace inf with NaN
    3) Drop rows with any remaining NaNs
    4) Align multiple DataFrames by shared (symbol, date) rows
    """
    cleaned = {}
    for name, df in dfs.items():
        # Select _yoy numeric columns
        yoy_cols = [c for c in df.select_dtypes(include=[float,int]).columns if c.endswith("_yoy")]
        # Keep only valid columns under nan_threshold
        valid_cols = [c for c in yoy_cols if df[c].isna().sum() <= nan_threshold]
        sub_df = df[valid_cols + keep_cols].copy()
        
        # Replace infs with NaN and drop rows with any remaining NaNs
        sub_df[valid_cols] = sub_df[valid_cols].replace([np.inf, -np.inf], np.nan)
        before = sub_df.shape[0]
        sub_df.dropna(subset=valid_cols, inplace=True)
        after = sub_df.shape[0]
        print(f"{name}: {before - after} rows dropped due to NaNs")
        
        # Create composite key
        sub_df["symbol_date"] = list(zip(sub_df["symbol"], sub_df["date"]))
        cleaned[name] = sub_df
        print(f"{name} shape after cleaning: {sub_df.shape}")
    
    # Find shared (symbol, date) across all dfs
    common_pairs = set.intersection(*(set(df["symbol_date"]) for df in cleaned.values()))
    print(f"Shared (symbol, date) rows across all dfs: {len(common_pairs)}")
    
    # Filter each df to shared rows
    aligned = {name: df[df["symbol_date"].isin(common_pairs)].copy().reset_index(drop=True) 
               for name, df in cleaned.items()}
    
    # Drop helper column
    for df in aligned.values():
        df.drop(columns=["symbol_date"], inplace=True)
    
    return aligned, common_pairs

In [14]:
aligned, common_pairs = clean_and_align({
    "income": income_growth,
    "balance": balance_growth,
    "cashflow": cashflow_growth
})

income: 504 rows dropped due to NaNs
income shape after cleaning: (502, 18)
balance: 504 rows dropped due to NaNs
balance shape after cleaning: (502, 13)
cashflow: 505 rows dropped due to NaNs
cashflow shape after cleaning: (501, 13)
Shared (symbol, date) rows across all dfs: 491


In [15]:
# Step 2: Run PCA on selected columns
def run_pca(df, selected_cols, n_components=5):
    sub_df = df[selected_cols].copy()
    scaled = StandardScaler().fit_transform(sub_df)
    pca = PCA(n_components=n_components)
    pcs = pca.fit_transform(scaled)
    pcs_df = pd.DataFrame(pcs, index=sub_df.index, columns=[f"PC{i+1}" for i in range(n_components)])
    
    print(f"PCA shape: {pcs_df.shape}")
    print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
    return pcs_df, pca

In [16]:
income_cols = ["revenue_yoy","netIncome_yoy","operatingIncomeRatio_yoy"]
balance_cols = ["totalAssets_yoy","totalLiabilities_yoy","totalStockholdersEquity_yoy"]
cashflow_cols = ["operatingCashFlow_yoy","freeCashFlow_yoy","netCashProvidedByOperatingActivities_yoy"]

income_pcs, income_pca_model = run_pca(aligned["income"], income_cols, n_components=3)
balance_pcs, balance_pca_model = run_pca(aligned["balance"], balance_cols, n_components=3)
cashflow_pcs, cashflow_pca_model = run_pca(aligned["cashflow"], cashflow_cols, n_components=3)

PCA shape: (491, 3)
Explained variance ratio: [0.34710423 0.34377133 0.30912445]
PCA shape: (491, 3)
Explained variance ratio: [0.646486  0.3168122 0.0367018]
PCA shape: (491, 3)
Explained variance ratio: [6.80308141e-01 3.19691688e-01 1.71405147e-07]


In [17]:
# Merge the three PCA results on symbol/date
merged_pca = income_pca.merge(balance_pca, on=["symbol","date"], suffixes=("_inc", "_bal"))
merged_pca = merged_pca.merge(cashflow_pca, on=["symbol","date"], suffixes=("", "_cf"))

# Get only PCA columns
pca_cols = [c for c in merged_pca.columns if c.startswith("PCA")]

# Compute correlation matrix
corr_all = merged_pca[pca_cols].corr()

# Display as heatmap
plt.figure(figsize=(6, 4))
sns.heatmap(corr_all, annot=False, cmap="coolwarm", center=0)
plt.title("Cross-Statement PCA Correlation Heatmap")
plt.show()


NameError: name 'income_pca' is not defined

In [ ]:
# --- 2. Merge with PCA DataFrame ---
# Example: merging with income PCA; can merge balance/cashflow similarly
merged_df = income_pca.merge(
    price_earnings_df[["symbol", "log_PE"]],
    on="symbol",
    how="inner"
)

# If you want, merge balance and cashflow PCA as well
merged_df = merged_df.merge(balance_pca, on=["symbol","date"], suffixes=("","_bal"))
merged_df = merged_df.merge(cashflow_pca, on=["symbol","date"], suffixes=("","_cf"))

# --- 3. Preview ---
print(merged_df.head())
print("Shape of merged dataset:", merged_df.shape)

In [ ]:
# Example: columns in merged_df
# 'PCA1', 'PCA1_bal', 'PCA1_cf', 'log_PE'

# --- 1. Separate regressions for each PCA1 ---
def run_ols_plot(x_col, y_col='log_PE', df=merged_df, color='blue', label=''):
    X = sm.add_constant(df[x_col])  # add intercept
    y = df[y_col]
    model = sm.OLS(y, X).fit()
    print(f"OLS Regression: {y_col} ~ {x_col}")
    print(model.summary())
    
    # Scatter + regression line
    sns.scatterplot(x=x_col, y=y_col, data=df, color=color, alpha=0.7, label=label)
    # Add regression line
    x_vals = np.linspace(df[x_col].min(), df[x_col].max(), 100)
    y_vals = model.params[0] + model.params[1]*x_vals
    plt.plot(x_vals, y_vals, color=color, linestyle='--')
    return model

plt.figure(figsize=(12,6))
model_income = run_ols_plot('PCA1', color='blue', label='Income PCA1')
model_balance = run_ols_plot('PCA1_bal', color='green', label='Balance PCA1')
model_cashflow = run_ols_plot('PCA1_cf', color='red', label='Cashflow PCA1')

plt.xlabel('PCA1 Component')
plt.ylabel('Log(Price / Earnings)')
plt.title('Log(PE) vs PCA1 for Each Statement with OLS Line')
plt.legend()
plt.grid(True)
plt.show()

# --- 2. Multiple regression including all three PCA1s ---
X_multi = merged_df[['PCA1','PCA1_bal','PCA1_cf']]
X_multi = sm.add_constant(X_multi)
y_multi = merged_df['log_PE']

multi_model = sm.OLS(y_multi, X_multi).fit()
print("Multiple regression: log_PE ~ PCA1 + PCA1_bal + PCA1_cf")
print(multi_model.summary())
